In [20]:
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor
import os

In [21]:
# Download training data from open datasets.
dataset_root_path = os.path.join(os.path.expanduser("~"), "Datasets")

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))  # Normalize the images to [-1, 1]
])

training_data = datasets.FashionMNIST(
    root=dataset_root_path,
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root=dataset_root_path,
    train=False,
    download=True,
    transform=ToTensor(),
)

In [22]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [23]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


In [24]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [25]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [26]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [27]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [28]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.297921  [   64/60000]
loss: 2.285061  [ 6464/60000]
loss: 2.267697  [12864/60000]
loss: 2.265239  [19264/60000]
loss: 2.253643  [25664/60000]
loss: 2.222579  [32064/60000]
loss: 2.241020  [38464/60000]
loss: 2.201527  [44864/60000]
loss: 2.200845  [51264/60000]
loss: 2.190714  [57664/60000]
Test Error: 
 Accuracy: 42.8%, Avg loss: 2.169447 

Epoch 2
-------------------------------
loss: 2.175487  [   64/60000]
loss: 2.170328  [ 6464/60000]
loss: 2.111295  [12864/60000]
loss: 2.130999  [19264/60000]
loss: 2.097296  [25664/60000]
loss: 2.031726  [32064/60000]
loss: 2.073481  [38464/60000]
loss: 1.988483  [44864/60000]
loss: 1.992541  [51264/60000]
loss: 1.951858  [57664/60000]
Test Error: 
 Accuracy: 56.5%, Avg loss: 1.928513 

Epoch 3
-------------------------------
loss: 1.951113  [   64/60000]
loss: 1.935081  [ 6464/60000]
loss: 1.811776  [12864/60000]
loss: 1.857598  [19264/60000]
loss: 1.765335  [25664/60000]
loss: 1.700122  [32064/600

In [16]:
model_path = os.path.join(os.path.expanduser("~"), "Models", "fashion_mnist.pth")

In [15]:
torch.save(model.state_dict(), model_path)
print(f"Saved PyTorch Model State to {model_path}")

Saved PyTorch Model State to /Users/anton/Models/fashion_mnist.pth


In [17]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load(model_path, weights_only=True))

<All keys matched successfully>

In [18]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


In [32]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.dropout1 = nn.Dropout(0.25)  # Dropout after conv layers
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.dropout2 = nn.Dropout(0.5)  # Dropout after fully connected layer
        self.fc2 = nn.Linear(128, 10)
    
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.dropout1(x)  # Apply dropout after convolutional layers
        x = x.view(-1, 64 * 7 * 7)  # Flatten the tensor
        x = torch.relu(self.fc1(x))
        x = self.dropout2(x)  # Apply dropout after the first fully connected layer
        x = self.fc2(x)
        return x

In [33]:
model = SimpleCNN().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [36]:
epochs = 50
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.156423  [   64/60000]
loss: 0.219439  [ 6464/60000]
loss: 0.076210  [12864/60000]
loss: 0.283038  [19264/60000]
loss: 0.154572  [25664/60000]
loss: 0.297373  [32064/60000]
loss: 0.248819  [38464/60000]
loss: 0.265889  [44864/60000]
loss: 0.236396  [51264/60000]
loss: 0.274124  [57664/60000]
Test Error: 
 Accuracy: 91.7%, Avg loss: 0.231828 

Epoch 2
-------------------------------
loss: 0.124240  [   64/60000]
loss: 0.225014  [ 6464/60000]
loss: 0.079173  [12864/60000]
loss: 0.317916  [19264/60000]
loss: 0.134249  [25664/60000]
loss: 0.232954  [32064/60000]
loss: 0.137080  [38464/60000]
loss: 0.249699  [44864/60000]
loss: 0.159286  [51264/60000]
loss: 0.177430  [57664/60000]
Test Error: 
 Accuracy: 91.4%, Avg loss: 0.245669 

Epoch 3
-------------------------------
loss: 0.122279  [   64/60000]
loss: 0.148368  [ 6464/60000]
loss: 0.100633  [12864/60000]
loss: 0.228612  [19264/60000]
loss: 0.115710  [25664/60000]
loss: 0.341340  [32064/600